### Semantic Kernel Agent using MCP Tools via Azure API Management

Model Context Protocol with Azure API Management to enable plug & play of tools for a Semantic Kernel Agent
![flow](../utils/media/model-context-protocol.gif)

0️⃣ Initialize Modules

In [32]:
import asyncio
from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.contents import ChatHistorySummarizationReducer
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPSsePlugin
from dotenv import load_dotenv
import os

load_dotenv(override=True)

apim_resource_gateway_url=os.getenv("APIM_RESOURCE_GATEWAY_URL")
apim_subscription_key=os.getenv("APIM_SUBSCRIPTION_KEY")
openai_api_version=os.getenv("AZURE_OPENAI_API_VERSION")               
openai_deployment_name=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")

1️⃣ Create the Semantic Kernel Orchestrator

In [41]:
async def main(user_input: str):
    # 1. Create the agent
    async with MCPSsePlugin(
        name="AgentPlugin",
        url=f"{apim_resource_gateway_url}/agent/sse",
        description="Menu Agent Plugin",
    ) as agent_plugin, \
    MCPSsePlugin(
        name="AnotherPlugin",
        url=f"{apim_resource_gateway_url}/weather/sse",
        description="Weather Plugin",
    ) as weather_plugin:
              
        orchestrator = ChatCompletionAgent(
            service=AzureChatCompletion(
                endpoint=apim_resource_gateway_url,
                api_key=apim_subscription_key,
                api_version=openai_api_version,                
                deployment_name=openai_deployment_name
            ),
            name="Orchestrator",
            instructions="Route the request to the appropriate agent.",
            plugins=[agent_plugin, weather_plugin],
        )

        thread: ChatHistoryAgentThread | None = None

        print(f"# User: {user_input}")
        # 2. Invoke the agent for a response
        response = await orchestrator.get_response(messages=user_input, thread=thread)
        print(f"# {response.name}: {response} ")
        thread = response.thread  # type: ignore

        # 3. Cleanup: Clear the thread
        await thread.delete() if thread else None


In [40]:
user_input = "What are special itens in the menu?"
await main(user_input)

  + Exception Group Traceback (most recent call last):
  |   File "c:\Users\pablocastao\OneDrive - Microsoft\Work\Clients\promigas\gbbai-semantickernel-using-mcptools\.venv\Lib\site-packages\semantic_kernel\connectors\mcp.py", line 203, in connect
  |     transport = await self._exit_stack.enter_async_context(self.get_mcp_client())
  |                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  |   File "C:\Users\pablocastao\AppData\Local\Programs\Python\Python311\Lib\contextlib.py", line 635, in enter_async_context
  |     result = await _enter(cm)
  |              ^^^^^^^^^^^^^^^^
  |   File "C:\Users\pablocastao\AppData\Local\Programs\Python\Python311\Lib\contextlib.py", line 204, in __aenter__
  |     return await anext(self.gen)
  |            ^^^^^^^^^^^^^^^^^^^^^
  |   File "c:\Users\pablocastao\OneDrive - Microsoft\Work\Clients\promigas\gbbai-semantickernel-using-mcptools\.venv\Lib\site-packages\mcp\client\sse.py", line 43, in sse_client
  |     async wi

In [35]:
user_input = "What is the weather in lisbon?"
await main(user_input)

  + Exception Group Traceback (most recent call last):
  |   File "c:\Users\pablocastao\OneDrive - Microsoft\Work\Clients\promigas\gbbai-semantickernel-using-mcptools\.venv\Lib\site-packages\semantic_kernel\connectors\mcp.py", line 203, in connect
  |     transport = await self._exit_stack.enter_async_context(self.get_mcp_client())
  |                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  |   File "C:\Users\pablocastao\AppData\Local\Programs\Python\Python311\Lib\contextlib.py", line 635, in enter_async_context
  |     result = await _enter(cm)
  |              ^^^^^^^^^^^^^^^^
  |   File "C:\Users\pablocastao\AppData\Local\Programs\Python\Python311\Lib\contextlib.py", line 204, in __aenter__
  |     return await anext(self.gen)
  |            ^^^^^^^^^^^^^^^^^^^^^
  |   File "c:\Users\pablocastao\OneDrive - Microsoft\Work\Clients\promigas\gbbai-semantickernel-using-mcptools\.venv\Lib\site-packages\mcp\client\sse.py", line 43, in sse_client
  |     async wi